In [ ]:
import pandas as pd
from google import genai
from google.genai.errors import APIError
import os
import time

# --- PENTING: Inisialisasi Gemini Client ---
# Pastikan variabel lingkungan GEMINI_API_KEY sudah diatur
try:
    client = genai.Client()
except Exception as e:
    print("❌ ERROR: Gagal menginisialisasi Gemini Client.")
    print("Pastikan Anda sudah mengatur GEMINI_API_KEY di lingkungan Anda.")
    exit()

# 1. Muat Data
file_path = "Untitled spreadsheet - Pelabelan data.csv"
try:
    df = pd.read_csv(file_path)
    # Ganti nilai NaN di kolom 'teks_ulasan' dengan string kosong agar tidak error
    df['teks_ulasan'] = df['teks_ulasan'].fillna('')
    # Hapus baris di mana 'teks_ulasan' kosong setelah pengisian NaN
    df = df[df['teks_ulasan'].str.strip() != '']
    
    if df.empty:
        print("Data kosong atau semua kolom 'teks_ulasan' kosong setelah dibersihkan.")
        exit()
        
except FileNotFoundError:
    print(f"❌ ERROR: File tidak ditemukan di jalur: {file_path}")
    exit()
except Exception as e:
    print(f"❌ ERROR: Gagal memuat atau membersihkan data: {e}")
    exit()

# 2. Fungsi untuk Mendapatkan Sentimen dari Gemini API
def get_sentiment_from_gemini(review_text):
    # Prompt yang jelas dan terstruktur untuk LLM
    prompt = f"""
    Analisis sentimen dari ulasan berikut. Berikan HANYA SATU KATA sebagai output: Positif, Negatif, atau Netral.
    
    Ulasan: "{review_text}"
    
    Sentimen:
    """
    
    try:
        # Panggil Gemini API
        response = client.models.generate_content(
            model='gemini-2.5-flash', # Model yang cepat dan efisien untuk tugas ini
            contents=prompt,
            config=genai.types.GenerateContentConfig(
                # Atur suhu ke rendah untuk hasil yang lebih deterministik (kurang kreatif)
                temperature=0.0
            )
        )
        
        # Bersihkan dan standarisasi output
        sentiment_label = response.text.strip().replace('.', '').capitalize()
        
        # Verifikasi output harus salah satu dari 3 label
        if sentiment_label in ['Positif', 'Negatif', 'Netral']:
            return sentiment_label
        else:
            # Jika output model tidak sesuai format, kembalikan Netral
            print(f"⚠️ Peringatan: Output Gemini tidak terstandarisasi untuk ulasan: '{review_text[:50]}...'. Output: '{response.text.strip()}'")
            return "Netral (Gagal Format)"
            
    except APIError as e:
        # Tangani error API (misalnya, batasan rate, otorisasi)
        print(f"❌ ERROR API untuk ulasan: '{review_text[:50]}...'. Error: {e}")
        return "Netral (Error API)"
    except Exception as e:
        # Tangani error lain
        print(f"❌ ERROR tak terduga untuk ulasan: '{review_text[:50]}...'. Error: {e}")
        return "Netral (Error Lain)"

# 3. Iterasi dan Aplikasikan Fungsi Pelabelan
sentiment_labels = []
# Hanya ambil 50 baris pertama untuk contoh agar cepat
sample_df = df.head(50) 
total_rows = len(sample_df)

print(f"\n⏳ Mulai pelabelan {total_rows} ulasan menggunakan Gemini API. Proses ini membutuhkan waktu...")
for index, row in sample_df.iterrows():
    review = row['teks_ulasan']
    label = get_sentiment_from_gemini(review)
    sentiment_labels.append(label)
    
    # Tampilkan progress
    current_progress = len(sentiment_labels)
    print(f"   Progress: {current_progress}/{total_rows} - Ulasan: '{review[:30]}...' -> Label: {label}")
    
    # Delay singkat untuk menghindari batasan rate limit pada API
    time.sleep(0.5)

# 4. Tambahkan Hasil ke DataFrame
sample_df['sentimen_label_gemini'] = sentiment_labels

# 5. Tampilkan Hasil
print("\n✅ Hasil Pelabelan Sentimen dengan Gemini (5 baris pertama):")
print(sample_df[['nama_penulis', 'rating', 'teks_ulasan', 'sentimen_label_gemini']].head())

# 6. Simpan Hasil ke File CSV Baru (Opsional)
sample_df.to_csv('data_terlabeli_gemini.csv', index=False)
print("\nData telah disimpan ke 'data_terlabeli_gemini.csv'")

❌ ERROR: Gagal menginisialisasi Gemini Client.
Pastikan Anda sudah mengatur GEMINI_API_KEY di lingkungan Anda.

⏳ Mulai pelabelan 50 ulasan menggunakan Gemini API. Proses ini membutuhkan waktu...
❌ ERROR tak terduga untuk ulasan: 'Mantap, laptop mereka second hand tapi baramg sepe...'. Error: name 'client' is not defined
   Progress: 1/50 - Ulasan: 'Mantap, laptop mereka second h...' -> Label: Netral (Error Lain)
❌ ERROR tak terduga untuk ulasan: 'Terbaik. Pilihan utama laptop secondhand berkualit...'. Error: name 'client' is not defined
   Progress: 2/50 - Ulasan: 'Terbaik. Pilihan utama laptop ...' -> Label: Netral (Error Lain)
❌ ERROR tak terduga untuk ulasan: 'Beli laptop disini udah yang ke 2x nya udah beli 3...'. Error: name 'client' is not defined
   Progress: 3/50 - Ulasan: 'Beli laptop disini udah yang k...' -> Label: Netral (Error Lain)
❌ ERROR tak terduga untuk ulasan: 'Pengalaman belanja di Hero bener-bener memuaskan. ...'. Error: name 'client' is not defined
   Progress: 4